In [1]:
import json
#from google.colab import files # solo para google collab
from google.cloud import storage
import pandas as pd
print(storage.__version__)

2.6.0


In [2]:
# google collab
#path_api_key = '/content/extreme-unison-399121-cadd77c555ca.json'

# entorno local
path_api_key = '/Users/manuel/gdrive/prgrmmng/gcloud_api_key/extreme-unison-399121-cadd77c555ca.json'

# extraigo las credenciales para el acceso
service_account_info = json.load(open(path_api_key))

# autorizo el acceso a cloud storage
client_storage = storage.Client.from_service_account_info(service_account_info)

In [3]:
# carga de datos
# unicamente para .py en VM:
#data_yelp_reviews_norm = pd.read_parquet('gs://1_transform/data_yelp_reviews_norm.parquet')

# para entorno Google Colab o maquina local:

# cambiar segun bucket
bucket = client_storage.get_bucket('1_transform')

# cambiar segun dataframe
data_blob = bucket.blob('data_yelp_reviews_norm.parquet')
data_yelp_reviews = pd.read_parquet(storage.fileio.BlobReader(data_blob))
data_yelp_reviews.info()
data_yelp_reviews.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415754 entries, 0 to 415753
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   review_id        415754 non-null  object 
 1   user_id          415754 non-null  object 
 2   business_id      415754 non-null  object 
 3   rating           415754 non-null  float64
 4   user_text        415754 non-null  object 
 5   user_time        415754 non-null  object 
 6   user_time_year   415754 non-null  int32  
 7   user_time_month  415754 non-null  int32  
 8   user_time_day    415754 non-null  int32  
 9   user_time_hms    415754 non-null  object 
 10  state_name       415754 non-null  object 
 11  state_code       415754 non-null  object 
 12  zcta5_geoid      415754 non-null  int64  
dtypes: float64(1), int32(3), int64(1), object(8)
memory usage: 36.5+ MB


,review_id,user_id,business_id,rating,user_text,user_time,user_time_year,user_time_month,user_time_day,user_time_hms,state_name,state_code,zcta5_geoid
0,igUar2sMmvX7Ps1AbvQXgg,MgTOBGcFm-koJmi9cg2JYw,2ZLIyUiQ-TQjG3nyfKDnxA,1.0,This is the worst Walmart ever conceived by hu...,2020-07-07 22:59:20,2020,7,7,22:59:20,missouri,MO,63303
1,gj2AHRvhP5kn8T5n2dmdrw,7Y2qxhOXVw-1qq09SMWzBA,aDd5-AAWriJscf2dTdGOCA,5.0,I love the food and the owner is awesome and w...,2020-07-18 22:00:09,2020,7,18,22:00:09,florida,FL,33578


In [7]:
columns_interes = [
    "state_name","state_code","zcta5_geoid",
    "user_id","business_id",
    "user_time_year","user_time_month","user_time_day",
    "rating"]

yelp_kpi_popularidad = data_yelp_reviews[
    columns_interes].sort_values(
        by=[
            "state_name","state_code","zcta5_geoid","business_id",
            "user_time_year","user_time_month","user_time_day"]
        ).copy() # type: ignore
yelp_kpi_popularidad.info()
yelp_kpi_popularidad.head()

<class 'pandas.core.frame.DataFrame'>
Index: 415754 entries, 238547 to 412931
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   state_name       415754 non-null  object 
 1   state_code       415754 non-null  object 
 2   zcta5_geoid      415754 non-null  int64  
 3   user_id          415754 non-null  object 
 4   business_id      415754 non-null  object 
 5   user_time_year   415754 non-null  int32  
 6   user_time_month  415754 non-null  int32  
 7   user_time_day    415754 non-null  int32  
 8   rating           415754 non-null  float64
dtypes: float64(1), int32(3), int64(1), object(4)
memory usage: 27.0+ MB


,state_name,state_code,zcta5_geoid,user_id,business_id,user_time_year,user_time_month,user_time_day,rating
238547,arizona,AZ,85614,hN9W8Sg6S1jcV0zmBHEoeA,0LMUH2jK3F5-wDT3zQ8Ffw,2021,7,8,1.0
276416,arizona,AZ,85614,lb6GPCR-ecVuOC57TIgYNg,0LMUH2jK3F5-wDT3zQ8Ffw,2021,8,11,1.0
347176,arizona,AZ,85614,Vctf3MEEP9oYOwkrO34snQ,0LMUH2jK3F5-wDT3zQ8Ffw,2021,10,9,2.0
180647,arizona,AZ,85614,oAenM-Xo_mBBP7neS7NKpA,1lSj24GO6VMJ2uB6hW3pGA,2020,12,28,5.0
217260,arizona,AZ,85614,UrL7w5UXlGsHLw1pNc4OxQ,1lSj24GO6VMJ2uB6hW3pGA,2021,7,8,1.0


In [15]:
# confirmamos que no hay duplicados:
print(len(data_yelp_reviews.loc[data_yelp_reviews.duplicated()]))

# mismo user_id y business_id pero diferente review_id
print(len(yelp_kpi_popularidad.loc[yelp_kpi_popularidad.duplicated()]))

# se mantiene el original sin remover duplicados aparentes
print(len(yelp_kpi_popularidad) == len(data_yelp_reviews))

0
11
True


In [16]:
yelp_kpi_popularidad_rating_business_id = yelp_kpi_popularidad.groupby(
    by=[
        "state_name","state_code","zcta5_geoid",
        "business_id","user_time_year","user_time_month"
        ],
    dropna=False
    )["rating"].size().reset_index().rename(columns={"rating":"business_id_rating_size"})
yelp_kpi_popularidad_rating_business_id["business_id_size_pctchange"] = yelp_kpi_popularidad_rating_business_id.groupby(
    by=[
        "state_name","state_code","zcta5_geoid",
        "business_id","user_time_year"
        ],
    dropna=False
    )["business_id_rating_size"].pct_change(1)*100
yelp_kpi_popularidad_rating_business_id.head()

,state_name,state_code,zcta5_geoid,business_id,user_time_year,user_time_month,business_id_rating_size,business_id_size_pctchange
0,arizona,AZ,85614,0LMUH2jK3F5-wDT3zQ8Ffw,2021,7,1,NaN
1,arizona,AZ,85614,0LMUH2jK3F5-wDT3zQ8Ffw,2021,8,1,0.0
2,arizona,AZ,85614,0LMUH2jK3F5-wDT3zQ8Ffw,2021,10,1,0.0
3,arizona,AZ,85614,1lSj24GO6VMJ2uB6hW3pGA,2020,12,1,NaN
4,arizona,AZ,85614,1lSj24GO6VMJ2uB6hW3pGA,2021,7,2,NaN


In [17]:
yelp_kpi_popularidad_rating_zcta5_geoid = yelp_kpi_popularidad.groupby(
     by=[
        "state_name","state_code","zcta5_geoid",
        "user_time_year","user_time_month"
        ],
    dropna=False
    )["rating"].size().reset_index().rename(columns={"rating":"zcta5_geoid_rating_size"})
yelp_kpi_popularidad_rating_zcta5_geoid["zcta5_geoid_size_pctchange"] = yelp_kpi_popularidad_rating_zcta5_geoid.groupby(
    by=[
        "state_name","state_code","zcta5_geoid",
        "user_time_year"
        ],
    dropna=False
    )["zcta5_geoid_rating_size"].pct_change(1)*100
yelp_kpi_popularidad_rating_zcta5_geoid.head()

,state_name,state_code,zcta5_geoid,user_time_year,user_time_month,zcta5_geoid_rating_size,zcta5_geoid_size_pctchange
0,arizona,AZ,85614,2020,7,17,NaN
1,arizona,AZ,85614,2020,8,16,-5.882353
2,arizona,AZ,85614,2020,9,17,6.250000
3,arizona,AZ,85614,2020,10,17,0.000000
4,arizona,AZ,85614,2020,11,17,0.000000


In [18]:
yelp_kpi_popularidad_crecimiento = pd.merge(
    left=yelp_kpi_popularidad_rating_business_id.drop(columns=["business_id_rating_size"]),
    right=yelp_kpi_popularidad_rating_zcta5_geoid.drop(columns=["zcta5_geoid_rating_size"]),
    how='left'
).dropna()
yelp_kpi_popularidad_crecimiento["kpi_popularidad_crecimiento"] = yelp_kpi_popularidad_crecimiento[
    "business_id_size_pctchange"] - yelp_kpi_popularidad_crecimiento["zcta5_geoid_size_pctchange"]
yelp_kpi_popularidad_crecimiento.drop(
    columns=["business_id_size_pctchange","zcta5_geoid_size_pctchange"], inplace=True)
yelp_kpi_popularidad_crecimiento.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119054 entries, 1 to 182744
Data columns (total 7 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   state_name                   119054 non-null  object 
 1   state_code                   119054 non-null  object 
 2   zcta5_geoid                  119054 non-null  int64  
 3   business_id                  119054 non-null  object 
 4   user_time_year               119054 non-null  int32  
 5   user_time_month              119054 non-null  int32  
 6   kpi_popularidad_crecimiento  119054 non-null  float64
dtypes: float64(1), int32(2), int64(1), object(3)
memory usage: 6.4+ MB


In [19]:
yelp_kpi_popularidad_parte = pd.merge(
    left=yelp_kpi_popularidad_rating_business_id.drop(columns=["business_id_size_pctchange"]),
    right=yelp_kpi_popularidad_rating_zcta5_geoid.drop(columns=["zcta5_geoid_size_pctchange"]),
    how='left'
).dropna()
yelp_kpi_popularidad_parte["kpi_popularidad_parte"] = (
    yelp_kpi_popularidad_parte["business_id_rating_size"] / yelp_kpi_popularidad_parte["zcta5_geoid_rating_size"]
    )*100
yelp_kpi_popularidad_parte.drop(
    columns=["business_id_rating_size","zcta5_geoid_rating_size"], inplace=True)
yelp_kpi_popularidad_parte.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182745 entries, 0 to 182744
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   state_name             182745 non-null  object 
 1   state_code             182745 non-null  object 
 2   zcta5_geoid            182745 non-null  int64  
 3   business_id            182745 non-null  object 
 4   user_time_year         182745 non-null  int32  
 5   user_time_month        182745 non-null  int32  
 6   kpi_popularidad_parte  182745 non-null  float64
dtypes: float64(1), int32(2), int64(1), object(3)
memory usage: 8.4+ MB


In [20]:
yelp_kpi_popularidad = pd.merge(
    left=yelp_kpi_popularidad_parte,
    right=yelp_kpi_popularidad_crecimiento,
    how='left')
yelp_kpi_popularidad.info()
yelp_kpi_popularidad.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182745 entries, 0 to 182744
Data columns (total 8 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   state_name                   182745 non-null  object 
 1   state_code                   182745 non-null  object 
 2   zcta5_geoid                  182745 non-null  int64  
 3   business_id                  182745 non-null  object 
 4   user_time_year               182745 non-null  int32  
 5   user_time_month              182745 non-null  int32  
 6   kpi_popularidad_parte        182745 non-null  float64
 7   kpi_popularidad_crecimiento  119054 non-null  float64
dtypes: float64(2), int32(2), int64(1), object(3)
memory usage: 9.8+ MB


,state_name,state_code,zcta5_geoid,business_id,user_time_year,user_time_month,kpi_popularidad_parte,kpi_popularidad_crecimiento
0,arizona,AZ,85614,0LMUH2jK3F5-wDT3zQ8Ffw,2021,7,3.571429,NaN
1,arizona,AZ,85614,0LMUH2jK3F5-wDT3zQ8Ffw,2021,8,4.761905,25.000000
2,arizona,AZ,85614,0LMUH2jK3F5-wDT3zQ8Ffw,2021,10,5.000000,13.043478
3,arizona,AZ,85614,1lSj24GO6VMJ2uB6hW3pGA,2020,12,5.000000,NaN
4,arizona,AZ,85614,1lSj24GO6VMJ2uB6hW3pGA,2021,7,7.142857,NaN
